## Normal Chain

In [5]:
from json import load
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.6,
)
parser = StrOutputParser()

template = PromptTemplate.from_template("You are a helpful assistant. Answer the question: {question}")

chain = template | llm | parser

result = chain.invoke({"question": "What is the capital of Kenya?"})
print(result)

The capital of Kenya is **Nairobi**.


In [6]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


## Sequential Chain

In [4]:
from json import load
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.6,
)
parser = StrOutputParser()

seq_template1 = PromptTemplate.from_template("""
Give me a short summary on topic : {topic}
""")

seq_template2 = PromptTemplate.from_template("""
Based on the summary, give me 5 key points on it \n {summary}
""")

seq_chain = seq_template1 | llm | parser | seq_template2 | llm | parser

seq_result = seq_chain.invoke({"topic": "Generative AI"})

print(seq_result)

Here are 5 key points from the summary about Generative AI:

1.  **Creation of New Content:** Generative AI's core function is to produce entirely new and original outputs.
2.  **Learning from Existing Data:** These AI systems learn patterns and structures by analyzing large datasets of existing content.
3.  **Mimicking or Inspired by Data:** The generated content is designed to resemble or be influenced by the data it was trained on.
4.  **Beyond Analysis:** Unlike analytical AI, generative AI goes beyond simply understanding information to actively creating it.
5.  **Diverse Applications:** It can generate various forms of content, including text, images, music, and code.


In [5]:
seq_chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *      

## Parallel Chain

In [7]:
from json import load
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from dotenv import load_dotenv

load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.6,
)
parser = StrOutputParser()

par_template1 = PromptTemplate.from_template("""
Give me a short summary on topic : {topic}
""")

par_template2 = PromptTemplate.from_template("""
Based on the summary, give me 5 key points on it \n {topic}
""")

par_chain1 = par_template1 | llm | parser
par_chain2 = par_template2 | llm | parser

parallel_chain = RunnableParallel(
    {
        "summary": par_chain1,
        "key_points": par_chain2

    }
)

mer_template = PromptTemplate.from_template("""
Combine the following information into a comprehensive output:

Summary: {summary}
Key Points: {key_points}
""")

mer_chain = mer_template | llm | parser

final_chain = parallel_chain | mer_chain

par_result = final_chain.invoke({"topic": "Generative AI"})

print(par_result)

Here's a comprehensive output combining the provided information:

**Comprehensive Output:**

Generative AI is a transformative type of artificial intelligence distinguished by its ability to **create new content**. This content can take various forms, including **text, images, music, and code**, all generated by the AI based on the intricate patterns and vast amounts of information it has assimilated from extensive datasets.

Crucially, generative AI differs from traditional AI approaches that primarily focus on analyzing or classifying existing data. Instead, generative AI is designed to **produce original outputs**. These outputs often exhibit a remarkable degree of human-likeness and can be entirely novel, pushing the boundaries of what AI can achieve.

**Key Points to Analyze (based on the provided summary):**

1.  **Content Creation:** Generative AI's core capability is the creation of new content.
2.  **Diverse Output Forms:** The content generated can span multiple modalities s

In [8]:
final_chain.get_graph().print_ascii()

                +-----------------------------------+                  
                | Parallel<summary,key_points>Input |                  
                +-----------------------------------+                  
                       ***                   ***                       
                   ****                         ****                   
                 **                                 **                 
    +----------------+                          +----------------+     
    | PromptTemplate |                          | PromptTemplate |     
    +----------------+                          +----------------+     
             *                                           *             
             *                                           *             
             *                                           *             
+------------------------+                  +------------------------+ 
| ChatGoogleGenerativeAI |                  | ChatGoogleGenerati

## Conditional Chain

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv

load_dotenv()

llm = ChatGroq(
    model="llama-3.1-8b-instant",  # free + fast
    temperature=0
)

class ReviewSentiment(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"] = Field(description="The sentiment of the review")

str_parser = StrOutputParser()
pydantic_parser = PydanticOutputParser(pydantic_object=ReviewSentiment)

con_template1 = PromptTemplate.from_template("""
Give me sentiment of the movie review {review} \n
format_instruction: {format_instruction}
""",
partial_variables={"format_instruction": pydantic_parser.get_format_instructions()}
)

review_chain = con_template1 | llm | pydantic_parser

#print(review_chain.invoke({"review": "This movie was fantastic! The acting was superb and the plot was engaging."}))

pos_template = PromptTemplate.from_template("""
The review is positive so write a short appreciation: \n {review}
""")

neg_template = PromptTemplate.from_template("""
The review is negative so write a short critique: \n {review}
""")

neu_template = PromptTemplate.from_template("""
The review is neutral so write a short balanced remarks: \n {review}
""")

default_template = PromptTemplate.from_template("""
I could not detect the clear sentiment. Provide a neutral response: \n {review}
""")

branch_chain = RunnableBranch(
    (lambda x: x.sentiment == "positive", pos_template | llm | str_parser),
    (lambda x: x.sentiment == "negative", neg_template | llm | str_parser),
    (lambda x: x.sentiment == "neutral", neu_template | llm | str_parser),
    default_template | llm | str_parser
)

final_con_chain = review_chain | branch_chain

review = "It was okay - some good moments, but nothing partucularly memorable. Just an average watch."

print(review_chain.invoke({"review": review}))

final_con_result = final_con_chain.invoke({"review": review})

print(final_con_result)

NameError: name 'PromptTemplate' is not defined